In [1]:
try:  # import as appropriate for 2.x vs. 3.x
    import tkinter as tk
    import tkinter.messagebox as tkMessageBox
except:
    import Tkinter as tk
    import tkMessageBox
   
"""
--------------------------------------------------------------------------------------------------------------------------
----------------------------------------------CLASSE DE L'ALIEN-----------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------
"""
class Alien(object):
    def __init__(self):
        self.id = None
        self.alive = True
    def touched_by(self, canvas, projectile):
        self.alive = False #mettre le booléen qui indique si l'alien est en vie à faux
        canvas.delete(self.id) #supprimer le canvas de l'alien touché
        canvas.delete(projectile) #supprimer le projectile
       
    def install_in(self, canvas, x, y, image, tags):
        self.id = canvas.create_image(x, y, image=image, tags=tags) #creer l'image de l'alien dans le canvas
       
    def move_in(self, canvas, dx, dy):
        canvas.move(self.id,dx,dy) #faire bouger l'alien de la distance dx sur l'horizentale et dy sur la verticale


"""
--------------------------------------------------------------------------------------------------------------------------
----------------------------------------------CLASSE DE LA FLOTTE D'ALIEN-------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------
"""
class Fleet(object):
    def __init__(self):
        self.aliens_lines = 5
        self.aliens_columns = 10
        self.aliens_inner_gap = 20
        self.alien_x_delta = 5
        self.alien_y_delta = 15
        fleet_size = self.aliens_lines * self.aliens_columns
        self.aliens_fleet = []
        self.image=tk.PhotoImage(file='alien.gif')
        self.image_game_over=tk.PhotoImage(file='gameover.gif')
        self.tag="image"
       
    def install_in(self, canvas):
        x = 50 #coordonnée x du 1er alien
        y = 50 #coordonnée y du 1er alien
        for i in range(self.aliens_lines) :
            for j in range(self.aliens_columns):
                a = Alien () #on créer une instance de la classe Alien
                self.aliens_fleet.append(a) #On ajoute cette instance a la liste
                a.install_in(canvas, x, y, self.image, self.tag) #on l'affiche avec la fonction install_in de la classe Alien
                x += 85 #on incrémente x pour que le prochain alien est le même y mais un x différent
            #A la fin de la boucle j, on a une ligne d'alien avec j colones, ici, 10 aliens.
            #La boucle i permet de répété la création d'une ligne i fois, ici 5 fois
            x = 50 #on met le prochain alien au même x que le 1er alien
            y += 55 #on incrémente le y pour que le prochain alien pour qu'il se touve en dessous de la ligne d'avant
      
    def move_in(self, canvas):
        
            bbox = canvas.bbox(self.tag) #coordonnées de la flotte d'alien
            wid_can = int(canvas.cget('width')) # on récupère la largeur du canva dans la variable wid_can (width_canva)
            if (bbox[2] > wid_can and bbox is not None) : #si le x2 de la flotte d'alien est plus grande que la largeur du canva
                    self.alien_x_delta = -self.alien_x_delta #on change le sens de déplacement horizontale
                    for alien in self.aliens_fleet : #on parcours la liste d'alien
                        alien.move_in(canvas, 0, self.alien_y_delta) #on déplace verticalement chaque alien

            elif bbox[0] < 0 and bbox is not None : #sinon si le x1 de la flotte d'alien est inférieur à 0 (bord gauche du canva)
                    self.alien_x_delta = -self.alien_x_delta #on change le sens de déplacement horizontale
                    for alien in self.aliens_fleet : #on parcours la liste d'alien
                        alien.move_in(canvas, 0, self.alien_y_delta) #on déplace verticalement chaque alien
            if (len(self.aliens_fleet)!=0):
                for alien in self.aliens_fleet : #on parcours la liste d'alien
                    alien.move_in(canvas, self.alien_x_delta, 0) #on déplace horizontalement chaque alien
       
           
    def manage_touched_aliens_by(self,canvas,defender):
        val = 0
        copie_lali = self.aliens_fleet.copy() #copie de la liste des aliens dans copie_lali
       
        for alien in copie_lali: #on parcourt la liste copiée
            copie_lbul = defender.fired_bullets.copy() #copie de la liste des bullets dans copie_lbul
            for bullet in copie_lbul:#on parcourt la copie de la liste de bullets 
                print(bullet.id) 
                if canvas.bbox(bullet.id) != None: #si les coordonnées de la bullet existent (ne sont pas nulles)
                    x1, y1, x2, y2 = canvas.bbox(bullet.id) #on récupère coordonnées dans x1,x2,y1,y2

                    overlapped = canvas.find_overlapping(x1, y1, x2, y2) #on stock dans overlapped l'(les) identifiant(s) supperposés à la bullet
                    print(overlapped)#on affiche l'identifiant de la bullet courante
                    if (overlapped[0] == 1 ): #si c'est l'identifiant du defender, on passe
                        pass
                    elif (len(overlapped) >= 2): #s'il y a au moins une instance supperposée à la bullet
                        
                        alien_val = overlapped[0]#on recupere dans val l'id de cette instance
                        val = overlapped[-1]#on recupere l'id de l'alien
                        if (val == bullet.id and alien_val == alien.id):#on teste si les id recuperés correspondent aux instances dans les listes de bullets et d'alien
                            alien.touched_by(canvas, bullet.id)#on supprime l'alien correspondant
                            defender.fired_bullets.remove(bullet)#on supprime la bullet correspondante
                        
   
           
    def get_width(self):
        return self.aliens_columns * (self.image.width() + self.aliens_inner_gap) - self.aliens_inner_gap#on retourne la taille du tag de l'ensemble des aliens
   
"""
--------------------------------------------------------------------------------------------------------------------------
----------------------------------------------CLASSE DU DEFENDEUR---------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------
"""
class Defender(object):
    def __init__(self):       
        self.width = 20
        self.height = 20
        self.move_delta = 20
        self.id = None
        self.max_fired_bullets = 8
        self.fired_bullets = []

    def install_in(self, canvas):
        x1 = int(canvas.cget('width')) // 2 - self.width// 2 # x1 du défendeur = moitié largeur fenetre - moitié largeur défendeur
        x2 = int(canvas.cget('width')) // 2 + self.width// 2 # x2 du défendeur = moitié largeur fenetre + moitié largeur défendeur
        y1 = int(canvas.cget('height')) - self.height # y1 du défendeur = moitié hauteur fenetre - moitié hauteur défendeur
        y2 = int(canvas.cget('height')) # y1 du défendeur = moitié hauteur fenetre
        self.id = canvas.create_rectangle(x1,y1,x2,y2,fill='red')

    def move_in(self,canvas, dx):
        canvas.move(self.id,dx,0) # déplacement de dx sur l'axe horizontale et de 0 sur l'axe verticale
       
    def fire(self, canvas):
        if len(self.fired_bullets)<8: # au maximum 8 tirs de balles autorisés
            bullet = Bullet(self) #création d'une instance de balle
            self.fired_bullets.append(bullet) #ajout de l'instance à la liste
            bullet.install_in(canvas) #affichage de la balle

           
           
"""
--------------------------------------------------------------------------------------------------------------------------
-----------------------------------------------CLASSE DE LA BALLE---------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------
"""   
class Bullet(object):
    def __init__(self, shooter):
        self.radius = 5
        self.color = "red" 
        self.speed = 8
        self.id = None
        self.shooter = shooter
       
    def install_in(self, canvas):
        x1 = canvas.coords(self.shooter.id)[0] + self.shooter.width // 2 - self.radius #x1 balle = x1 défendeur + moitié largeur défendeur - rayon balle
        x2 = canvas.coords(self.shooter.id)[2] - self.radius #x2 balle = x2 défendeur - rayon balle
        y1 = int(canvas.cget('height')) - self.shooter.height - self.radius  * 2 -5# y1 balle = hauteur fenêtre - hauteur défendeur - diametre balle
        y2 = int(canvas.cget('height')) - self.shooter.height - 5 #y2 balle = hauteur fenêtre - hauteur défendeur
        self.id = canvas.create_oval(x1,y1,x2,y2,fill='red')

    def move_in(self, canvas):
        if canvas.bbox(self.id) != None:
            bbox = canvas.bbox(self.id) #coordonné de la balle récupéré dans "bbox"
            if bbox[3]-self.speed > 0 : #si la balle est dans la fenêtre
                canvas.move(self.id,0,-self.speed) #alors elle se déplace
            else :
                canvas.delete(self.id) #sinon on la supprime
                self.shooter.fired_bullets.remove(self) #tout en la suprrimant du tableau contenant les balles
           
       
   
"""
--------------------------------------------------------------------------------------------------------------------------
-----------------------------------------------CLASSE DU JEU---------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------
Ici, le canva est créé.
"""   
class Game(object):
    def __init__(self, frame):
        self.frame = frame
        self.fleet = Fleet()
        self.defender = Defender()
        self.height =600
        self.width = self.fleet.get_width() * 1.25
        self.canvas = tk.Canvas(self.frame, width=self.width, height=self.height, bg="#000000",highlightthickness=0)
        self.canvas.pack(side="top")
        self.defender.install_in(self.canvas)
        self.fleet.install_in(self.canvas)
       
    def animation(self):
        self.move_aliens_fleet()#on appelle la fonction qui bouge la flotte des aliens
        self.move_bullets()#on appelle la fonction qui bouge chaque bullet
        self.canvas.after(30,self.animation)#on insere un temps de latence 
       
    def start_animation(self):
        self.canvas.after(10,self.animation)
       
    def move_bullets(self):
        for bullet in self.defender.fired_bullets:#on parcourt la liste des bullets du defender
            bullet.move_in(self.canvas)#on applique la fonction move sur chaque bullet
           
    def move_aliens_fleet(self):
        self.fleet.move_in(self.canvas)#on appelle la fonction move sur le tag de tous les aliens
        self.fleet.manage_touched_aliens_by(self.canvas,self.defender)#appel de la fonction de collision sur le tag des aliens et les bullets du defender
       
    def keypress(self, event):
        dx=0
        bbox = self.canvas.bbox(self.defender.id)#recuperation des coordonnées du defender
        if (event.keysym == 'Left') and ( bbox[0] > 0 ):#si la touche gauche est appuyée est le coin gauche du defender est dans le canvas
            dx=-10 #on peut se déplacer de 10 vers la gauche
        elif (event.keysym == 'Right') and (bbox[2] < self.width):#si la touche droite est appuyée est le coin droit du defender est dans le canvas
            dx=+10#on peut se déplacer de 10 vers la droite
        self.defender.move_in(self.canvas,dx) #application de la fonction move sur le defender
        if event.keysym == 'space':
            self.defender.fire(self.canvas)#si l'espace est appuyé le defender tire une bullet
       
"""
--------------------------------------------------------------------------------------------------------------------------
-----------------------------------------------CLASSE SPACE INVADERS------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------
Ici, la fenêtre est crée.
"""  
class SpaceInvaders(object):
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Space Invaders")
        self.frame = tk.Frame(self.root)
        self.frame.pack(side="top")
        self.game = Game(self.frame)
       
    def play(self):
        self.game.start_animation()
        self.root.bind("<Key>", self.game.keypress)
        self.root.mainloop()
                
SpaceInvaders().play() #lancement du jeu

52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
52
(52,)
53
(53,)
52
(52,)
53
(53,)
52
(52,)
53
(53,)
52
(52,)
53
(53,)
52
(52,)
53
(53,)
52
(52,)
5

(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)


(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56

(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
5

(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
52
(52,)
53


...

(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(

(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
53
(53,)
54
(54,)
55
(55, 56, 5

(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54
(54,)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
54

55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 56, 57)
55
(55, 56, 57)
56
(55, 56, 57)
57
(55, 

(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55, 56)
55
(55, 56)
56
(55,